<a href="https://colab.research.google.com/github/peremartra/Apress_LLProjects_Book/blob/main/P1-NL2SQL/6_3_AWS_Bedrock_NL2SQL_Client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>6.3-Calling AWS Bedrock from Python. </h2>
    <h3></h3>
    <p>by <b>Pere Martra</b></p>
</div>



In this notebook, we make a call to a Model from AWS Bedrock  that we've set up to work as a translator for SQL queries from natural language.


In [1]:
pip install -q boto3==1.34.108

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import json
from getpass import getpass

In [3]:
aws_access_key_id = getpass('AWS Acces key: ')

AWS Acces key:  ········


In [4]:
aws_secret_access_key = getpass('AWS Secret Key: ')

AWS Secret Key:  ········


In [5]:
client = boto3.client("bedrock-runtime",
                      region_name="us-west-2",
                      aws_access_key_id = aws_access_key_id,
                      aws_secret_access_key= aws_secret_access_key)


In [6]:
# Set the model ID, e.g., Llama 3 8B Instruct.
model_id = "meta.llama3-8b-instruct-v1:0"

In [7]:
# Define the user message to send.
user_message = "What is the name of the best paid employee?"

In [8]:
model_instructions = """
Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word to appropriately answer the question.
- **Return Only SQL Code. 
   ### Input
   Generate a SQL query that answers the question below.
   This query will run on a database whose schema is represented in this string:

   create table employees(
       ID_Usr INT primary key,-- Unique Id for employee
       name VARCHAR -- Name of employee
       );

   create table salary(
       ID_Usr INT,-- Unique Id for employee
       year DATE, -- Date
       salary FLOAT, --Salary of employee
       foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
       );

   create table studies(
       ID_study INT, -- Unique ID study
       ID_Usr INT, -- ID employee
       educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
       Institution VARCHAR, --Name of instituon where eployee studied
       Years DATE, -- Date acomplishement stdy
       Speciality VARCHAR, -- Speciality of studies
       primary key (ID_study, ID_Usr), --Primary Key ID_Usr + ID_Study
       foreign key(ID_Usr) references employees (ID_Usr)
       );

"""


In [9]:
# Embed the message in Llama 3's prompt format.
prompt = f"""
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
{model_instructions}
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{user_message}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

In [10]:
print (prompt)


<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>

Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word to appropriately answer the question.
- **Return Only SQL Code. 
   ### Input
   Generate a SQL query that answers the question below.
   This query will run on a database whose schema is represented in this string:

   create table employees(
       ID_Usr INT primary key,-- Unique Id for employee
       name VARCHAR -- Name of employee
       );

   create table salary(
       ID_Usr INT,-- Unique Id for employee
       year DATE, -- Date
       salary FLOAT, --Salary of employee
       foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
       );

   create table studies(
       ID_study INT, -- Unique ID study
       ID_Usr INT, -- ID employee
       educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
       Insti

In [27]:
# Format the request payload using the model's native structure.
hyper = {
    "prompt": prompt,
    # Optional inference parameters:
    "max_gen_len": 512,
    "temperature": 0.0
}

In [28]:
# Encode and send the request.
response = client.invoke_model(body=json.dumps(hyper), modelId=model_id)

In [29]:
# Decode the native response body.
model_response = json.loads(response["body"].read())

In [30]:
# Extract and print the generated text.
response_text = model_response["generation"]
print(response_text)

Here is the SQL query that answers the question:

```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_Usr = s.ID_Usr
ORDER BY s.salary DESC
LIMIT 1;
```
